# 03.11 - Working with Time Series

Pandas is able to work with several formats of date and time. We can categorize them as:

1. Time stamps 
2. Time intervals
3. Time deltas

### Dates and Times in Python

Let's have a look at what is already available in Python and their relationship with Pandas:

#### Native Python dates and times: <code>datetime</code> and <code>dateutil</code>

Using built-in <code>datetime</code>, we can manually build a date:

In [1]:
from datetime import datetime
datetime(year=2015, month=7, day=4)

datetime.datetime(2015, 7, 4, 0, 0)

Using the <code>dateutil</code> module, you can parse dates from a variety of string formats:

In [2]:
from dateutil import parser
date = parser.parse("4th of July, 2015")
date

datetime.datetime(2015, 7, 4, 0, 0)

Or printing the day of the week:

In [4]:
date.strftime('%A')

'Saturday'